The goal of this code was to use data measured from the motor cortex of monkeys to predict the direction they moved their arm during a task. We used a poisson model and maximum likelihood information to achieve this. We used the pre motor signals to predict, during movement signals to predict, and lastly both signals to predict.
Download data at: https://drive.google.com/file/d/1yxycfrB3JVFf8eN_nYH9svqN-yDcFYAM/view?usp=sharing

In [4]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [5]:
# Load data
data = np.load('ReachData.npz',allow_pickle=True)
r = data['r']
cfr = data['cfr']-1

In [8]:
num_trials = 1127
num_neurons = 190
num_directions = 8
num_training_trials = 50
num_test_trials = num_trials - (num_directions*num_training_trials)
neurons = list(range(0, 190))

#Define times
timeTouchHelds = np.asarray([o.timeTouchHeld for o in r])
timeGoCues = np.asarray([o.timeGoCue for o in r])
timeTargetAcquires = np.asarray([o.timeTargetAcquire for o in r])
#key times for each trial so length:1127

#Window Boundaries
plan_window = np.vstack((timeTouchHelds, timeGoCues))
movement_window = np.vstack((timeGoCues, timeTargetAcquires))
combined_window = np.vstack((timeTouchHelds, timeTargetAcquires))

#Window lengths in seconds
plan_window_len = (np.diff(plan_window, axis=0)/1000)[0]
movement_window_len = (np.diff(movement_window, axis=0)/1000)[0]
combined_window_len = (np.diff(combined_window, axis=0)/1000)[0]

#Initialize count matrices
pw_cnt = np.zeros((num_trials, num_neurons))
mw_cnt = np.zeros((num_trials, num_neurons))
cw_cnt = np.zeros((num_trials, num_neurons))

rand_neurons = np.random.choice(neurons, num_neurons, replace=False)
#In windows, normalize to count per second
for trial in range(num_trials):
    iteration = 0
    for neuron in rand_neurons:
        spikes = r[trial].unit[neuron].spikeTimes
        pw_cnt[trial, iteration] = np.sum((plan_window[0,trial] <= spikes) & (spikes <= plan_window[1,trial]))/plan_window_len[trial]
        mw_cnt[trial, iteration] = np.sum((movement_window[0,trial] <= spikes) & (spikes <= movement_window[1,trial]))/movement_window_len[trial]
        cw_cnt[trial, iteration] = np.sum((combined_window[0,trial] <= spikes) & (spikes <= combined_window[1,trial]))/combined_window_len[trial]
        iteration += 1
    iteration = 0
training_trials = np.zeros((num_directions,num_training_trials), dtype=int)
p_mu = np.zeros((num_neurons,num_directions))
m_mu = np.zeros((num_neurons,num_directions))
c_mu = np.zeros((num_neurons,num_directions))

for direction in range(num_directions):
    #Find trials with current direction, then randomly select 50 for training
    trials_with_same_direction = np.where(cfr == direction)[0]
    training_trials[direction,:] = random.sample(trials_with_same_direction.tolist(), num_training_trials)

    #Estimate Parameters
    p_mu[:,direction] = np.transpose(np.mean(pw_cnt[training_trials[direction,:],:], axis=0))
    m_mu[:, direction] = np.transpose(np.mean(mw_cnt[training_trials[direction, :], :], axis=0))
    c_mu[:, direction] = np.transpose(np.mean(cw_cnt[training_trials[direction, :], :], axis=0))
#Replace zeros
p_mu[p_mu==0] = np.finfo(float).eps
m_mu[m_mu==0] = np.finfo(float).eps
c_mu[c_mu==0] = np.finfo(float).eps

#Initialize arrays
poisson_plan = np.zeros((num_test_trials,num_directions))
poisson_move = np.zeros((num_test_trials,num_directions))
poisson_comb = np.zeros((num_test_trials,num_directions))
test_directions = np.zeros((num_test_trials))
training_trials_vec = np.reshape(training_trials, (-1,num_directions*num_training_trials))[0]

#Test Trials
test_trial_index = 0
for trial in range(num_trials):
    if not trial in training_trials_vec:

        #Retrieve test direction
        test_directions[test_trial_index] = cfr[trial]

        #Test counts
        pw_cnt_test = np.transpose(pw_cnt[trial,:])
        mw_cnt_test = np.transpose(mw_cnt[trial, :])
        cw_cnt_test = np.transpose(cw_cnt[trial, :])

        #Compute log likelihoods for each direction
        for direction in range(num_directions):
            #Poisson likelihoods
            poisson_plan[test_trial_index, direction] = -np.sum(p_mu[:, direction]) + np.dot(np.log(p_mu[:, direction]), pw_cnt_test)
            poisson_move[test_trial_index, direction] = -np.sum(m_mu[:, direction]) + np.dot(np.log(m_mu[:, direction]), mw_cnt_test)
            poisson_comb[test_trial_index, direction] = -np.sum(c_mu[:, direction]) + np.dot(np.log(c_mu[:, direction]), cw_cnt_test)

        #Increment index
        test_trial_index = test_trial_index + 1

#Compute accuracy
ppw_acc = 100*np.sum(np.argmax(poisson_plan, axis=1) == test_directions)/num_test_trials
pmw_acc = 100*np.sum(np.argmax(poisson_move, axis=1) == test_directions)/num_test_trials
pcw_acc = 100*np.sum(np.argmax(poisson_comb, axis=1) == test_directions)/num_test_trials

print('---Poisson---')
print("Plan: %.2f%%"%ppw_acc)
print("Move: %.2f%%"%pmw_acc)
print("Combined: %.2f%%"%pcw_acc)

---Poisson---
Plan: 93.40%
Move: 97.11%
Combined: 99.72%
